In [1]:
import torch

In [2]:
# Inport squad dataset from train-v2.0.json
# Read json file
import json
with open('train-v2.0.json') as f:
	train_data = json.load(f)

In [3]:
paragraphs = []
for article in train_data['data']:
	for para in article['paragraphs']:
		paragraphs.append(para['context'])

In [4]:
print(paragraphs[0])

Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".


In [37]:
# Get questions and answers
questions = []
answers = []
question_ids = []
question_map = {}
for article in train_data['data']:
    for para in article['paragraphs']:
        tmp_questions = []
        tmp_answers = []
        tmp_question_ids = []
        for qa in para['qas']:
            tmp_question_ids.append(qa['id'])
            try:
                question_map[qa['id']] = [qa['question'],
                                          para['context'], qa['answers'][0]['text']]
                tmp_questions.append(qa['question'])
                tmp_answers.append(qa['answers'][0]['text'])
            except:
                print("No answers for: {}".format(qa['question']))
                tmp_questions.append("")
                tmp_answers.append("")
        questions.append(tmp_questions)
        answers.append(tmp_answers)
        question_ids.append(tmp_question_ids)


No answers for: What category of game is Legend of Zelda: Australia Twilight?
No answers for: What consoles can be used to play Australia Twilight?
No answers for: When was Australia Twilight launched in North America?
No answers for: When could GameCube owners purchase Australian Princess?
No answers for: What year was the Legend of Zelda: Australian Princess originally planned for release?
No answers for: What land does Ocarina serve to protect?
No answers for: What character helped Link in Ocarina Princess?
No answers for: Who is the protagonist of Midna of Time?
No answers for: From what alternate dimension does this dimension take place from?
No answers for: When does the Waker take place?
No answers for: What accolade did Radar Princess receive after its release?
No answers for: Under which brand was Radar Princess for the Nintendo Wii published?
No answers for: When is the HD version of Radar Princess slated for launch?
No answers for: At the time of release what episode of Lege

In [6]:
# Get question ids
# question_ids = []
# for article in train_data['data']:
# 	for para in article['paragraphs']:
# 		# tmp_question_ids = []
# 		for qa in para['qas']:
# 			question_ids.append(qa['id'])
		# question_ids.append(tmp_question_ids)

In [34]:
print(len(question_map))

86821


In [8]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [9]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [10]:
question = "How many parameters does BERT-large have?"
answer_text = "BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance."

In [11]:
# Apply the tokenizer to the input text, treating them as a text-pair.
input_ids = tokenizer.encode(question, answer_text)

print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 70 tokens.


In [12]:
# BERT only needs the token IDs, but for the purpose of inspecting the 
# tokenizer's behavior, let's also get the token strings and display them.
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')

[CLS]           101
how           2,129
many          2,116
parameters   11,709
does          2,515
bert         14,324
-             1,011
large         2,312
have          2,031
?             1,029

[SEP]           102

bert         14,324
-             1,011
large         2,312
is            2,003
really        2,428
big           2,502
.             1,012
.             1,012
.             1,012
it            2,009
has           2,038
24            2,484
-             1,011
layers        9,014
and           1,998
an            2,019
em            7,861
##bed         8,270
##ding        4,667
size          2,946
of            1,997
1             1,015
,             1,010
02            6,185
##4           2,549
,             1,010
for           2,005
a             1,037
total         2,561
of            1,997
340          16,029
##m           2,213
parameters   11,709
!               999
altogether   10,462
it            2,009
is            2,003
1             1,015
.             1,01

In [13]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [14]:
# Run our example through the model.
outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                             token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                             return_dict=True) 

start_scores = outputs.start_logits
end_scores = outputs.end_logits


In [15]:
# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens[answer_start:answer_end+1])

print('Answer: "' + answer + '"')

Answer: "340 ##m"


In [16]:
# Start with the first token.
answer = tokens[answer_start]

# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
    
    # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
    
    # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]

print('Answer: "' + answer + '"')

Answer: "340m"


In [39]:
# Convert paragraph and question to tokens
tokenized_questions_map = {}
# def tokenize_paragraph(i):
for question in question_map:
	# print(question)
	tokenized_questions_map[question] = tokenizer.encode(question_map[question][0], question_map[question][1])
